<a href="https://colab.research.google.com/github/summerolmstead/Interest-Rates/blob/main/Effective_Federal_Funds_Rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Summer Olmstead

Part 3 of Final Project Data Visualization

Macroeconomic Finance Theme

In [1]:
import pandas_datareader.data as web
from datetime import datetime

#same date range used with all
start = datetime(2010, 1, 1)
end = datetime(2025, 4, 1)

#loading federal funds rate data same as previous but specifically fedfunds which is interest rate
fed_rate = web.DataReader('FEDFUNDS', 'fred', start, end)

#have to reset index and rename for consistency
fed_rate.reset_index(inplace=True)
fed_rate.rename(columns={'FEDFUNDS': 'Interest Rate'}, inplace=True) # the fed funds rate is the benchmark US Interest Rate

In [2]:
fed_rate.describe() #structure

,DATE,Interest Rate
count,183,183.000000
mean,2017-07-31 23:44:15.737704960,1.280109
min,2010-01-01 00:00:00,0.050000
25%,2013-10-16 12:00:00,0.095000
50%,2017-08-01 00:00:00,0.190000
75%,2021-05-16 12:00:00,1.910000
max,2025-03-01 00:00:00,5.330000
std,NaN,1.748296


In [3]:
fed_rate.shape #dimensions

(183, 2)

In [4]:
fed_rate.head()

,DATE,Interest Rate
0,2010-01-01,0.11
1,2010-02-01,0.13
2,2010-03-01,0.16
3,2010-04-01,0.20
4,2010-05-01,0.20


In [5]:
#lets use plotly again to visualize a simple range slider plot with a slider once again
import plotly.express as px

fig = px.line(
    fed_rate,
    x='DATE',
    y='Interest Rate',
    title='Federal Funds Effective Interest Rate (2010–2025)',
    labels={'DATE': 'Date', 'Interest Rate': 'Interest Rate (%)'}
)

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [6]:
import plotly.graph_objects as go

#want to do recession shading so we will define recession periods ( ex: COVID-19)
recessions = [
    {'start': '2020-02-01', 'end': '2020-04-01', 'label': 'COVID-19'},
    {'start': '2022-03-01', 'end': '2023-07-01', 'label': 'Inflation Hike'} #the Federal Reserve's monetary tightening cycle in response to elevated inflation picked these dates as they went with curve
]

#first we have to calculate interest rate jumps (when month-to-month increase is > 0.5%)
fed_rate['Rate Change'] = fed_rate['Interest Rate'].diff()
spikes = fed_rate[fed_rate['Rate Change'] > 0.5]

#initalize figure
fig = go.Figure()

#first we do the line for Interest Rate (IR)
fig.add_trace(go.Scatter(
    x=fed_rate['DATE'],
    y=fed_rate['Interest Rate'],
    mode='lines',
    name='Interest Rate (%)',
    line=dict(color='darkgreen')
))

#now we have to add recession shading
for rec in recessions:
    fig.add_vrect(
        x0=rec['start'], x1=rec['end'],
        fillcolor="LightSalmon", opacity=0.4,
        layer="below", line_width=0,
        annotation_text=rec['label'], annotation_position="top left"
    )

#lets also add spike markers
fig.add_trace(go.Scatter(
    x=spikes['DATE'],
    y=spikes['Interest Rate'],
    mode='markers',
    marker=dict(color='red', size=8, symbol='triangle-up'),
    name='Rate Hike Spike'
))

#final layout for graph altogher with time slider and everything
fig.update_layout(
    title='Federal Funds Rate (2010–2025) with Recession Shading and Rate Hike Spikes',
    xaxis=dict(title='Date', rangeslider_visible=True),
    yaxis=dict(title='Interest Rate (%)'),
    legend=dict(x=0.01, y=0.99)
)

fig.show()
fig.write_html('fed_rate_recession_spikes.html')


Context :

When interest rates rise, it's usually the Fed trying to slow down the economy. Higher rates make borrowing more expensive — so people take out fewer loans, credit card payments go up, and overall spending tends to cool off. On the business side, companies slow down investment too since financing becomes pricier. The goal is usually to curb inflation, but it can also lead to slower economic growth or even a recession if the brakes get pushed too hard. So these sharp jumps in interest rates often signal that the Fed is reacting to something intense, like rising inflation — and you can usually see the effects ripple across things like the stock market, employment, and consumer behavior.